In [5]:
import requests
import time

def get_conceptnet_triples(concept, limit=10, retries=3, sleep_time=1):
    url = f"https://api.conceptnet.io/query?node=/c/en/{concept}&limit={limit}"
    
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=20)
            
            # Check for successful response
            if response.status_code != 200:
                print(f"Warning: ConceptNet returned status {response.status_code}. Response: {response.text}")
                time.sleep(sleep_time)
                continue
            
            data = response.json()  # This is where JSONDecodeError happened before
            triples = []
            for edge in data.get("edges", []):
                head = edge["start"]["label"]
                rel = edge["rel"]["label"]
                tail = edge["end"]["label"]
                triples.append((head, rel, tail))
            
            return triples
        
        except requests.exceptions.RequestException as e:
            print(f"Network error: {e}")
            time.sleep(sleep_time)
        
        except ValueError:
            print("ConceptNet returned invalid JSON, retrying...")
            time.sleep(sleep_time)
    
    # If all retries fail
    print(f"Failed to fetch ConceptNet triples for '{concept}' after {retries} attempts.")
    return []


In [6]:
choices = ["ignore", "enforce", "authoritarian", "yell at", "avoid"]
triples = get_conceptnet_triples("punishing")

relevant_triples = [t for t in triples if any(c in t for c in choices)]

<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>

<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>

<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.14.0 (Ubuntu)</center>
</body>
</html>

Failed to fetch ConceptNet triples for 'punishing' after 3 attempts.


In [1]:
import requests
import urllib.parse
import networkx as nx
from collections import deque

BASE_URL = "https://api.conceptnet.io/query?node=/c/en/{}&other=/c/en/{}"
LOOKUP_URL = "https://api.conceptnet.io/c/en/{}"

def get_edges(term, max_edges=50):
    """Fetch edges for a given term from ConceptNet API."""
    term_enc = urllib.parse.quote(term.replace(" ", "_"))
    url = LOOKUP_URL.format(term_enc)
    edges = []
    try:
        data = requests.get(url, timeout=5).json()
        for e in data.get('edges', []):
            rel = e['rel']['label']
            start = e['start']['label']
            end = e['end']['label']
            weight = e.get('weight', 1.0)
            edges.append((start.lower(), end.lower(), {'rel': rel, 'weight': weight}))
            if len(edges) >= max_edges:
                break
    except Exception as ex:
        print("Error:", ex)
    return edges


def build_subgraph(seed, k=2, max_edges_per_node=50):
    """Build a k-hop subgraph from ConceptNet around the seed concept."""
    G = nx.Graph()
    visited = set()
    queue = deque([(seed, 0)])
    
    while queue:
        node, depth = queue.popleft()
        if depth >= k or node in visited:
            continue
        visited.add(node)
        
        edges = get_edges(node, max_edges_per_node)
        for u, v, attrs in edges:
            G.add_edge(u, v, **attrs)
            if v not in visited:
                queue.append((v, depth + 1))
    return G


def find_relation_path(G, source, target):
    """Find shortest path (if exists) between two nodes and return relation chain."""
    try:
        path = nx.shortest_path(G, source.lower(), target.lower())
        relations = []
        for i in range(len(path)-1):
            rel = G[path[i]][path[i+1]]['rel']
            relations.append(rel)
        return path, relations
    except (nx.NetworkXNoPath, nx.NodeNotFound):
        return None, []


In [2]:
# === Example usage ===
question_concept = "revolving door"
choices = ["bank", "library", "mall"]

# Build a subgraph around the question concept
G = build_subgraph(question_concept, k=2)

# Analyze connectivity to each choice
for c in choices:
    path, rels = find_relation_path(G, question_concept, c)
    if path:
        print(f"Path from '{question_concept}' to '{c}': {path}")
        print(f"Relations: {rels}\n")
    else:
        print(f"No path found between {question_concept} and {c}\n")


Error: Expecting value: line 1 column 1 (char 0)
No path found between revolving door and bank

No path found between revolving door and library

No path found between revolving door and mall



In [13]:
from gradio_client import Client

In [14]:
client = Client("cstr/conceptnet_normalized")

Loaded as API: https://cstr-conceptnet-normalized.hf.space ✔


In [20]:
result = client.predict(
	word="dog",
	lang="en",
	selected_relations=['RelatedTo', 'IsA', 'PartOf', 'HasA', 'UsedFor', 'CapableOf', 'AtLocation', 'Causes', 'HasSubevent', 'HasFirstSubevent', 'HasLastSubevent', 'HasPrerequisite', 'HasProperty', 'MotivatedByGoal', 'ObstructedBy', 'Desires', 'CreatedBy', 'Synonym', 'Antonym', 'DistinctFrom', 'DerivedFrom', 'SymbolOf', 'DefinedAs', 'MannerOf', 'LocatedNear', 'HasContext', 'SimilarTo', 'EtymologicallyRelatedTo', 'EtymologicallyDerivedFrom', 'CausesDesire', 'MadeOf', 'ReceivesAction', 'ExternalURL', 'NotDesires', 'NotUsedFor', 'NotCapableOf', 'NotHasProperty'],
	api_name="/get_semantic_profile"
)

In [21]:
result

"# 🧠 Semantic Profile: 'dog' (EN)\n\n**Node:** `http://conceptnet.io/c/en/dog` → **dog**\n\n## RelatedTo\n\n- **dog** RelatedTo → *pet* `[9.830]`\n- **dog** RelatedTo → *animal* `[9.410]`\n- **dog** RelatedTo → *canine* `[7.626]`\n- **dog** RelatedTo → *woof* `[4.396]`\n- **dog** RelatedTo → *bark* `[3.977]`\n- **dog** RelatedTo → *tail* `[3.891]`\n- **dog** RelatedTo → *wolf* `[3.879]`\n- *flea* RelatedTo → **dog** `[9.021]`\n- *animal* RelatedTo → **dog** `[3.129]`\n- *bone* RelatedTo → **dog** `[2.902]`\n- *pet* RelatedTo → **dog** `[2.418]`\n- *god* RelatedTo → **dog** `[2.077]`\n- *puppy* RelatedTo → **dog** `[2.075]`\n- *pup* RelatedTo → **dog** `[2.067]`\n\n## IsA\n\n- **dog** IsA → *loyal friend* `[6.633]`\n- **dog** IsA → *pet* `[6.000]`\n- **dog** IsA → *mammal* `[5.292]`\n- **dog** IsA → *canine* `[4.899]`\n- **dog** IsA → *good friend* `[3.464]`\n- **dog** IsA → *four legged animal* `[2.828]`\n- **dog** IsA → *curious observer of mankind* `[2.000]`\n- *poodle* IsA → **dog**

In [11]:
result = client.predict(
	api_name="/get_schema"
)
print(result)

# 📚 Schema (Normalized)

**Repo:** [cstr/conceptnet-normalized-multi](https://huggingface.co/datasets/cstr/conceptnet-normalized-multi)

**Schema:** Text URLs (`node_norm`, `rel_norm`) are stored once. The `edge_norm` table uses fast integer keys (`_fk`) for joins.

## Tables & Row Counts

- **node_norm:** 10,661,608 rows
- **rel_norm:** 50 rows
- **edge_norm:** 12,205,434 rows

## Indices

- **idx_start_fk:** `CREATE INDEX idx_start_fk ON edge_norm(start_fk)`
- **idx_end_fk:** `CREATE INDEX idx_end_fk ON edge_norm(end_fk)`
- **idx_rel_fk:** `CREATE INDEX idx_rel_fk ON edge_norm(rel_fk)`

## Common Relations (from `rel_norm`)

- **Antonym:** `http://conceptnet.io/r/Antonym`
- **AtLocation:** `http://conceptnet.io/r/AtLocation`
- **CapableOf:** `http://conceptnet.io/r/CapableOf`
- **Causes:** `http://conceptnet.io/r/Causes`
- **CausesDesire:** `http://conceptnet.io/r/CausesDesire`
- **CreatedBy:** `http://conceptnet.io/r/CreatedBy`
- **DefinedAs:** `http://conceptnet.io/r/DefinedAs`
- *

In [25]:
from gradio_client import Client
import networkx as nx
import re
from collections import deque

# ------------------------------
# 1. Query conceptnet_normalized
# ------------------------------
client = Client("cstr/conceptnet_normalized")

def get_conceptnet_profile(term, lang="en", relations=None):
    """Query the ConceptNet normalized model for a term."""
    if relations is None:
        relations = [
            'RelatedTo','IsA','PartOf','HasA','UsedFor','CapableOf','AtLocation',
            'Causes','HasSubevent','HasFirstSubevent','HasLastSubevent',
            'HasPrerequisite','HasProperty','MotivatedByGoal','ObstructedBy',
            'Desires','CreatedBy','Synonym','Antonym','DistinctFrom','DerivedFrom',
            'SymbolOf','DefinedAs','MannerOf','LocatedNear','HasContext','SimilarTo',
            'EtymologicallyRelatedTo','EtymologicallyDerivedFrom','CausesDesire',
            'MadeOf','ReceivesAction','ExternalURL','NotDesires','NotUsedFor',
            'NotCapableOf','NotHasProperty'
        ]
    
    result = client.predict(
        word=term,
        lang=lang,
        selected_relations=relations,
        api_name="/get_semantic_profile"
    )
    return result  # markdown string
        

# ------------------------------
# 2. Parse Markdown into triples
# ------------------------------
TRIPLE_REGEX = re.compile(
    r"- \*\*(.*?)\*\* ([A-Za-z]+) → \*(.*?)\* \`\[(.*?)\]\`"
)

def parse_semantic_profile(markdown_text):
    """
    Extract triples: (source, relation, target, weight)
    from the markdown returned by conceptnet_normalized.
    """
    triples = []
    for match in TRIPLE_REGEX.finditer(markdown_text):
        source, rel, target, weight = match.groups()
        triples.append((source.lower(), target.lower(), rel, float(weight)))
        
    return triples


# ------------------------------
# 3. Build Graph for a term
# ------------------------------
def build_graph_for_term(term):
    """Fetch ConceptNet normalized data and build a local graph."""
    md = get_conceptnet_profile(term)
    triples = parse_semantic_profile(md)

    G = nx.Graph()
    for src, tgt, rel, w in triples:
        G.add_edge(src, tgt, rel=rel, weight=w)
    return G


# ------------------------------
# 4. K-hop expansion
# ------------------------------
def build_k_hop_subgraph(seed_term, k=2):
    """
    Build a k-hop neighborhood graph around seed_term using 
    conceptnet_normalized instead of conceptnet.io API.
    """
    G = nx.Graph()
    visited = set()
    queue = deque([(seed_term, 0)])

    while queue:
        node, depth = queue.popleft()
        if depth > k or node in visited:
            continue

        visited.add(node)

        # build graph for this node
        local_graph = build_graph_for_term(node)
        G = nx.compose(G, local_graph)  # merge graphs

        # expand neighbors
        for neighbor in local_graph.neighbors(node):
            if neighbor not in visited:
                queue.append((neighbor, depth + 1))

    return G


# ------------------------------
# 5. Path finding
# ------------------------------
def find_relation_path(G, source, target):
    """
    Find shortest path + relation sequence.
    """
    try:
        path = nx.shortest_path(G, source.lower(), target.lower())
        rels = []
        for i in range(len(path) - 1):
            rels.append(G[path[i]][path[i+1]]["rel"])
        return path, rels
    except Exception:
        return None, []

# ------------------------------
# 6. Example Usage
# ------------------------------
if __name__ == "__main__":
    question_concept = "revolving door"
    choices = ["bank", "library", "mall"]

    print("Building k-hop graph...")
    G = build_k_hop_subgraph(question_concept, k=2)

    for c in choices:
        path, rels = find_relation_path(G, question_concept, c)
        if path:
            print(f"\nPath to {c}: {path}")
            print(f"Relations: {rels}")
        else:
            print(f"\nNo path found to {c}")


Loaded as API: https://cstr-conceptnet-normalized.hf.space ✔
Building k-hop graph...


NetworkXError: The node revolving door is not in the graph.

In [30]:
import re
import networkx as nx
from collections import deque
from gradio_client import Client

client = Client("cstr/conceptnet_normalized")

relations = [
            'RelatedTo','IsA','PartOf','HasA','UsedFor','CapableOf','AtLocation',
            'Causes','HasSubevent','HasFirstSubevent','HasLastSubevent',
            'HasPrerequisite','HasProperty','MotivatedByGoal','ObstructedBy',
            'Desires','CreatedBy','Synonym','Antonym','DistinctFrom','DerivedFrom',
            'SymbolOf','DefinedAs','MannerOf','LocatedNear','HasContext','SimilarTo',
            'EtymologicallyRelatedTo','EtymologicallyDerivedFrom','CausesDesire',
            'MadeOf','ReceivesAction','ExternalURL','NotDesires','NotUsedFor',
            'NotCapableOf','NotHasProperty'
        ]

###############################################
# 1) Fetch concept profile from ConceptNet model
###############################################
def get_conceptnet_profile(word, relations):
    return client.predict(
        word=word,
        lang="en",
        selected_relations=relations,
        api_name="/get_semantic_profile"
    )


###############################################
# 2) Parse markdown result → edges in a graph
###############################################
def parse_profile_to_edges(text, center_word):
    G = nx.DiGraph()
    center_word = center_word.lower()

    # Split into relation sections
    sections = re.split(r"## ", text)[1:]  # removes header part

    for sec in sections:
        lines = sec.strip().split("\n")
        header = lines[0].strip()
        relation = header  # e.g. "RelatedTo" or "IsA"

        # Each following line is an edge
        for line in lines[1:]:
            match = re.match(r"- (.*) " + re.escape(relation) + r" → (.*) \[", line)
            if not match:
                continue

            src_raw = match.group(1)
            tgt_raw = match.group(2)

            # clean source/target words
            src = re.sub(r"[*`]", "", src_raw).lower()
            tgt = re.sub(r"[*`]", "", tgt_raw).lower()

            G.add_edge(src, tgt, relation=relation)

    return G

def normalize_concept_name(concept):
    concept = concept.lower().strip()

    candidates = [
        concept,
        concept.replace(" ", "_"),
        concept.replace(" ", "-"),
    ]

    # also include individual tokens
    tokens = concept.split()
    if len(tokens) > 1:
        candidates.extend(tokens)

    return candidates

def find_valid_concept(concept, relations):
    candidates = normalize_concept_name(concept)

    for c in candidates:
        try:
            result = get_conceptnet_profile(c, relations)
            # detect empty/no-data profile
            if "No results" not in result and "Total relations: 0" not in result:
                return c, result
        except:
            pass

    return None, None

###############################################
# 3) Build K-hop graph using repeated calls to model
###############################################
def build_k_hop_graph(seed_word, k, relations):
    resolved, profile = find_valid_concept(seed_word, relations)
    if resolved is None:
        print(f"[Warning] No ConceptNet data for: {seed_word}")
        return nx.DiGraph()

    seed_word = resolved.lower()
    
    G = nx.DiGraph()
    visited = set()
    queue = deque([(seed_word.lower(), 0)])

    while queue:
        word, depth = queue.popleft()

        if depth > k or word in visited:
            continue

        visited.add(word)

        # Fetch conceptnet profile for this word
        try:
            profile_text = get_conceptnet_profile(word, relations)
        except Exception as e:
            print(f"[Warning] Could not fetch profile for: {word} ({e})")
            continue

        # Parse edges
        local_graph = parse_profile_to_edges(profile_text, word)

        # Merge with main graph
        G = nx.compose(G, local_graph)

        # Neighbor expansion for next hops
        if word in local_graph:
            for nbr in local_graph.neighbors(word):
                if nbr not in visited:
                    queue.append((nbr, depth + 1))

    return G


###############################################
# 4) Find relation path between two concepts
###############################################
def find_relation_path(G, src, tgt):
    src = src.lower()
    tgt = tgt.lower()

    if src not in G or tgt not in G:
        return None, None

    try:
        path = nx.shortest_path(G, src, tgt)
    except:
        return None, None

    relations = []
    for a, b in zip(path, path[1:]):
        relations.append(G[a][b]["relation"])

    return path, relations


###############################################
# 5) Example usage (CommonsenseQA)
###############################################
question_concept = "revolving door"
choices = ["bank", "library", "mall"]

print("Building k-hop graph... (this may take several seconds)")
G = build_k_hop_graph(question_concept, k=3, relations=relations)

print("\n--- Results ---")
for c in choices:
    path, rels = find_relation_path(G, question_concept, c)
    print(f"\nChoice: {c}")
    print("Path:", path)
    print("Relations:", rels)


Loaded as API: https://cstr-conceptnet-normalized.hf.space ✔
Building k-hop graph... (this may take several seconds)

--- Results ---

Choice: bank
Path: None
Relations: None

Choice: library
Path: None
Relations: None

Choice: mall
Path: None
Relations: None


In [31]:
get_conceptnet_profile("revolving", relations)

"# 🧠 Semantic Profile: 'revolving' (EN)\n\n**Node:** `http://conceptnet.io/c/en/revolving` → **revolving**\n\n## RelatedTo\n\n- *دوار* RelatedTo → **revolving** `[1.000]`\n- *drehbar* RelatedTo → **revolving** `[1.000]`\n- *calender* RelatedTo → **revolving** `[1.000]`\n- *centrepin* RelatedTo → **revolving** `[1.000]`\n- *circuit* RelatedTo → **revolving** `[1.000]`\n- *drawbore* RelatedTo → **revolving** `[1.000]`\n- *fleckerl* RelatedTo → **revolving** `[1.000]`\n\n## IsA\n\n*No results*\n\n## PartOf\n\n*No results*\n\n## HasA\n\n*No results*\n\n## UsedFor\n\n*No results*\n\n## CapableOf\n\n*No results*\n\n## AtLocation\n\n*No results*\n\n## Causes\n\n*No results*\n\n## HasSubevent\n\n*No results*\n\n## HasFirstSubevent\n\n*No results*\n\n## HasLastSubevent\n\n*No results*\n\n## HasPrerequisite\n\n*No results*\n\n## HasProperty\n\n- *credit* HasProperty → **revolving** `[1.000]`\n\n## MotivatedByGoal\n\n*No results*\n\n## ObstructedBy\n\n*No results*\n\n## Desires\n\n*No results*\n\

In [35]:
get_conceptnet_profile("revolving door", relations)

"# 🧠 Semantic Profile: 'revolving_door' (EN)\n\n**Node:** `http://conceptnet.io/c/en/revolving_door` → **revolving door**\n\n## RelatedTo\n\n- *drehtür* RelatedTo → **revolving_door** `[1.000]`\n- *revolving doors* RelatedTo → **revolving_door** `[1.000]`\n- *tourniquet* RelatedTo → **revolving_door** `[1.000]`\n- *bussola* RelatedTo → **revolving_door** `[1.000]`\n\n## IsA\n\n*No results*\n\n## PartOf\n\n*No results*\n\n## HasA\n\n*No results*\n\n## UsedFor\n\n- **revolving_door** UsedFor → *entering building* `[3.464]`\n- **revolving_door** UsedFor → *enter building* `[1.000]`\n- **revolving_door** UsedFor → *exiting building* `[1.000]`\n- **revolving_door** UsedFor → *getting into building* `[1.000]`\n\n## CapableOf\n\n*No results*\n\n## AtLocation\n\n- **revolving_door** AtLocation → *lobby* `[2.000]`\n- **revolving_door** AtLocation → *bank* `[1.000]`\n- **revolving_door** AtLocation → *building* `[1.000]`\n- **revolving_door** AtLocation → *department store* `[1.000]`\n- **revolv

In [36]:
print("# 🧠 Semantic Profile: 'revolving_door' (EN)\n\n**Node:** `http://conceptnet.io/c/en/revolving_door` → **revolving door**\n\n## RelatedTo\n\n- *drehtür* RelatedTo → **revolving_door** `[1.000]`\n- *revolving doors* RelatedTo → **revolving_door** `[1.000]`\n- *tourniquet* RelatedTo → **revolving_door** `[1.000]`\n- *bussola* RelatedTo → **revolving_door** `[1.000]`\n\n## IsA\n\n*No results*\n\n## PartOf\n\n*No results*\n\n## HasA\n\n*No results*\n\n## UsedFor\n\n- **revolving_door** UsedFor → *entering building* `[3.464]`\n- **revolving_door** UsedFor → *enter building* `[1.000]`\n- **revolving_door** UsedFor → *exiting building* `[1.000]`\n- **revolving_door** UsedFor → *getting into building* `[1.000]`\n\n## CapableOf\n\n*No results*\n\n## AtLocation\n\n- **revolving_door** AtLocation → *lobby* `[2.000]`\n- **revolving_door** AtLocation → *bank* `[1.000]`\n- **revolving_door** AtLocation → *building* `[1.000]`\n- **revolving_door** AtLocation → *department store* `[1.000]`\n- **revolving_door** AtLocation → *entrance to building* `[1.000]`\n- **revolving_door** AtLocation → *hotel lobby* `[1.000]`\n- **revolving_door** AtLocation → *mall* `[1.000]`\n\n## Causes\n\n*No results*\n\n## HasSubevent\n\n*No results*\n\n## HasFirstSubevent\n\n*No results*\n\n## HasLastSubevent\n\n*No results*\n\n## HasPrerequisite\n\n*No results*\n\n## HasProperty\n\n*No results*\n\n## MotivatedByGoal\n\n*No results*\n\n## ObstructedBy\n\n*No results*\n\n## Desires\n\n*No results*\n\n## CreatedBy\n\n*No results*\n\n## Synonym\n\n- *drehtür* Synonym → **revolving_door** `[1.000]`\n\n## Antonym\n\n*No results*\n\n## DistinctFrom\n\n*No results*\n\n## DerivedFrom\n\n*No results*\n\n## SymbolOf\n\n*No results*\n\n## DefinedAs\n\n*No results*\n\n## MannerOf\n\n*No results*\n\n## LocatedNear\n\n*No results*\n\n## HasContext\n\n*No results*\n\n## SimilarTo\n\n*No results*\n\n## EtymologicallyRelatedTo\n\n*No results*\n\n## EtymologicallyDerivedFrom\n\n*No results*\n\n## CausesDesire\n\n*No results*\n\n## MadeOf\n\n*No results*\n\n## ReceivesAction\n\n*No results*\n\n## ExternalURL\n\n*No results*\n\n## NotDesires\n\n*No results*\n\n## NotUsedFor\n\n*No results*\n\n## NotCapableOf\n\n*No results*\n\n## NotHasProperty\n\n*No results*\n\n---\n**Total relations:** 16")

# 🧠 Semantic Profile: 'revolving_door' (EN)

**Node:** `http://conceptnet.io/c/en/revolving_door` → **revolving door**

## RelatedTo

- *drehtür* RelatedTo → **revolving_door** `[1.000]`
- *revolving doors* RelatedTo → **revolving_door** `[1.000]`
- *tourniquet* RelatedTo → **revolving_door** `[1.000]`
- *bussola* RelatedTo → **revolving_door** `[1.000]`

## IsA

*No results*

## PartOf

*No results*

## HasA

*No results*

## UsedFor

- **revolving_door** UsedFor → *entering building* `[3.464]`
- **revolving_door** UsedFor → *enter building* `[1.000]`
- **revolving_door** UsedFor → *exiting building* `[1.000]`
- **revolving_door** UsedFor → *getting into building* `[1.000]`

## CapableOf

*No results*

## AtLocation

- **revolving_door** AtLocation → *lobby* `[2.000]`
- **revolving_door** AtLocation → *bank* `[1.000]`
- **revolving_door** AtLocation → *building* `[1.000]`
- **revolving_door** AtLocation → *department store* `[1.000]`
- **revolving_door** AtLocation → *entrance to buil